In [2]:
# load the necessary packages 

import os
import scipy
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

import plotly.express as px
import geopandas as gpd
from scipy.spatial import cKDTree

import imageio
import os
import io
import requests

import datetime as dt
from dateutil.relativedelta import *

import netCDF4
from netCDF4 import Dataset, num2date

import cmocean

import seaborn as sns
import pymannkendall as mk

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from matplotlib import pyplot as plt
%matplotlib inline

from PIL import Image  # To create gifs
from shapely.geometry import Point
from shapely.geometry import LineString


In [36]:
# loading the csv with Debbie's path 
df = pd.read_csv('debbie.csv',parse_dates=['datetime'])
time = df['datetime'].dt.tz_localize(None)
df

,seq,datetime,lon,wind_kts,wind_mps,year,pressure,lat,name
0,13,2017-03-22 00:00:00+00:00,152.00,14.967616,7.7,2016,1006,-13.20,DEBBIE
1,13,2017-03-22 06:00:00+00:00,151.70,14.967616,7.7,2016,1010,-12.00,DEBBIE
2,13,2017-03-22 12:00:00+00:00,154.00,14.967616,7.7,2016,1011,-12.10,DEBBIE
3,13,2017-03-23 00:00:00+00:00,152.60,20.021616,10.3,2016,1008,-13.30,DEBBIE
4,13,2017-03-23 06:00:00+00:00,151.60,20.021616,10.3,2016,1000,-14.10,DEBBIE
5,13,2017-03-23 12:00:00+00:00,151.50,20.021616,10.3,2016,1000,-14.50,DEBBIE
6,13,2017-03-23 18:00:00+00:00,151.30,25.075616,12.9,2016,1000,-15.30,DEBBIE
7,13,2017-03-24 00:00:00+00:00,151.30,25.075616,12.9,2016,999,-15.90,DEBBIE
8,13,2017-03-24 06:00:00+00:00,151.60,25.075616,12.9,2016,996,-16.40,DEBBIE
9,13,2017-03-24 12:00:00+00:00,151.80,29.935231,15.4,2016,996,-16.80,DEBBIE


In [38]:
# Creating a new dataset with only lat, lon and time for Debbie
path = df[['lon', 'lat', 'datetime']]  # Select the columns
# path = path.set_index('datetime')  # Set 'datetime' as the index
path

,lon,lat,datetime
0,152.00,-13.20,2017-03-22 00:00:00+00:00
1,151.70,-12.00,2017-03-22 06:00:00+00:00
2,154.00,-12.10,2017-03-22 12:00:00+00:00
3,152.60,-13.30,2017-03-23 00:00:00+00:00
4,151.60,-14.10,2017-03-23 06:00:00+00:00
5,151.50,-14.50,2017-03-23 12:00:00+00:00
6,151.30,-15.30,2017-03-23 18:00:00+00:00
7,151.30,-15.90,2017-03-24 00:00:00+00:00
8,151.60,-16.40,2017-03-24 06:00:00+00:00
9,151.80,-16.80,2017-03-24 12:00:00+00:00


In [40]:
# open the biogeochemical data from eReefs (netCDF file) 
month = 3
year = 2017
netCDF_datestr = str(year)+'-'+format(month, '02')

# GBR4 BIO
biofiles = ["https://thredds.ereefs.aims.gov.au/thredds/dodsC/GBR4_H2p0_B3p1_Cq3b_Dhnd/daily.nc"]

ds_bio = xr.open_mfdataset(biofiles)

# Define the time range (BEFORE the cycone only)
start_date = '2017-03-01'
end_date = '2017-03-22'

# Slice the dataset based on the time range
ds_bio_slice = ds_bio.sel(time=slice(start_date, end_date), k=-1)
ds_bio_slice

<xarray.Dataset>
Dimensions:          (latitude: 723, longitude: 491, time: 22)
Coordinates:
    zc               float64 dask.array<chunksize=(), meta=np.ndarray>
  * latitude         (latitude) float64 -28.7 -28.67 -28.64 ... -7.066 -7.036
  * longitude        (longitude) float64 142.2 142.2 142.2 ... 156.8 156.8 156.9
  * time             (time) datetime64[ns] 2017-03-01T02:00:00 ... 2017-03-22...
Data variables: (12/108)
    TOTAL_NITROGEN   (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    alk              (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    BOD              (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    Chl_a_sum        (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    CO32             (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    DetBL_N          (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    ...               ...
    SGH_N            (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    SGH_N_pr         (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    SGHROOT_N        (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    SGROOT_N         (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    TSSM             (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
    Zenith2D         (time, latitude, longitude) float32 dask.array<chunksize=(22, 723, 491), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                   CF-1.0
    NCO:                           netCDF Operators version 4.7.7 (Homepage =...
    RunID:                         2
    _CoordSysBuilder:              ucar.nc2.dataset.conv.CF1Convention
    aims_ncaggregate_buildDate:    2021-09-07T22:58:30+10:00
    aims_ncaggregate_datasetId:    products__ncaggregate__ereefs__GBR4_H2p0_B...
    ...                            ...
    metadata_link:                 https://eatlas.org.au/data/uuid/61f3a6df-2...
    paramfile:                     /home/bai155/EMS_solar2/gbr4_H2p0_B3p1_Cb/...
    paramhead:                     eReefs 4 km grid. SOURCE Catchments with 2...
    technical_guide_link:          https://eatlas.org.au/pydio/public/aims-er...
    technical_guide_publish_date:  2020-08-18
    title:                         eReefs AIMS-CSIRO GBR4 BioGeoChemical 3.1 ...

In [42]:
# creating a dataset with only temperature, chlorophyll-a and secchi for the 10 days of the cyclone 

# Select the variables of interest
selected_vars = ds_bio_slice[['Chl_a_sum', 'temp', 'Secchi']]

# Create a new dataset with these variables
new_bio = xr.Dataset(
    {
        'Chl_a_sum': selected_vars['Chl_a_sum'],
        'temp': selected_vars['temp'],
        'Secchi': selected_vars['Secchi']
    }
)

In [30]:
# Now that we have the IDs of all the data within 50km of the cyclone path, we need to compute the mean, max and min for each 
# of the selected variables (temp, secchi and chlorophyll-a). We will do that for before, during and after the cyclone. 

# Starting with BEFORE the 9 days of the cyclone (01/03/2017 - 22/03/2017)

In [49]:
# List of variables to process
variables = ['temp', 'Secchi', 'Chl_a_sum']

# Create a dictionary to store statistics for each variable and position
all_stats = {f'{var}_{pos}': {'Mean': [], 'Max': [], 'Min': []} for var in variables for pos in range(37) if not (25 <= pos <= 34)}

# Loop through each position file, excluding: 
# (1) pos0 - pos3 and pos37 to pos39 - because there is no data in these positions
# (2) pos25 to pos34 - because the cyclone is above land in these positions 

for pos in range(37):
    if pos in range(4) or 25 <= pos <= 34:
        continue
    
    # Read the corresponding CSV file
    idpos = pd.read_csv(f'idclosetocyc/pos{pos}.csv')
    
    # Loop through each variable
    for var in variables:
        # Creating lists to store mean, max, and min values for all days (0-21 corresponds to 22 days)
        daily_means = []
        daily_maxs = []
        daily_mins = []
        
        # Loop through each day (time index from 0 to 21)
        for day in range(22):
            # Extract values for the specific day and position
            values = getattr(new_bio, var).isel(time=day).values[idpos['idlat'].values, idpos['idlon'].values]
            
            # Calculate statistics, ignoring NaNs
            mean_val = np.nanmean(values)
            max_val = np.nanmax(values)
            min_val = np.nanmin(values)
            
            # Append results to the lists
            daily_means.append(mean_val)
            daily_maxs.append(max_val)
            daily_mins.append(min_val)
        
        # Store the statistics for the current variable and position
        all_stats[f'{var}_{pos}']['Mean'] = daily_means
        all_stats[f'{var}_{pos}']['Max'] = daily_maxs
        all_stats[f'{var}_{pos}']['Min'] = daily_mins

# Create a DataFrame combining mean, max, and min values for all variables
data = {}
for var in variables:
    for pos in range(37):
        if pos in range(4) or 25 <= pos <= 34:
            continue
        data[f'{var}_pos{pos}_Mean'] = all_stats[f'{var}_{pos}']['Mean']
        data[f'{var}_pos{pos}_Max'] = all_stats[f'{var}_{pos}']['Max']
        data[f'{var}_pos{pos}_Min'] = all_stats[f'{var}_{pos}']['Min']

# Create DataFrame
df_all_stats = pd.DataFrame(data, index=[f'Day {i+1}' for i in range(22)])

# Transpose DataFrame to have days as columns and variables as rows
df_all_stats_T = df_all_stats.T

# Save the transposed DataFrame to a new CSV file
df_all_stats_T.to_csv('combined_stats_BEFORE.csv')

print('Combined statistics for temp, Secchi, and Chl_a_sum saved to combined_stats_BEFORE.csv')


Combined statistics for temp, Secchi, and Chl_a_sum saved to combined_stats_BEFORE.csv
